<h1>Анализ лидов и продаж для онлайн-школы психологических профессий<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта-и-постановка-задач" data-toc-modified-id="Описание-проекта-и-постановка-задач-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта и постановка задач</a></span></li><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Загрузка-данных,-предобработка,-EDA" data-toc-modified-id="Загрузка-данных,-предобработка,-EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка данных, предобработка, EDA</a></span><ul class="toc-item"><li><span><a href="#Заявки-(лиды)" data-toc-modified-id="Заявки-(лиды)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Заявки (лиды)</a></span></li><li><span><a href="#Продажи" data-toc-modified-id="Продажи-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Продажи</a></span></li><li><span><a href="#Конверсии" data-toc-modified-id="Конверсии-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Конверсии</a></span></li></ul></li><li><span><a href="#Динамические-отчеты" data-toc-modified-id="Динамические-отчеты-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Динамические отчеты</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

Начало <a class="anchor" id="100-bullet"></a>

## Описание проекта и постановка задач

Заказчик: онлайн-школа психологических профессий.

Цель сайта: лидогенерация. Сбор лидов идет через форму "Оставить заявку", также можно написать в чат, в "подвале" сайта есть номер телефона и имейл.

Заказчиком предоставлен файл "Тестовое задание.xls" с двумя листами: "Заявки" и "Продажи".

Задачи:

1. Используя данные листа "Заявки", построить динамический отчет, где можно посмотреть данные в разрезе: Курс, Количество лидов, utm_sourse - за различный период (день, неделя, месяц).
2. Используя данные листа "Продажи", построить динамический отчет в разрезе: Курс, Менеджер, за различный период (день, неделя, месяц).
3. На основе обоих листов построить конверсию курсов из заявки в продажу и в зависимости от источников, с возможностью смотреть данные за различный период (день, неделя, месяц).

## Импорт библиотек

In [1]:
import pandas as pd
from datetime import date, time, timedelta
pd.options.display.max_rows = 500

import plotly.express as px
from plotly import graph_objects as go

[Вернуться в начало](#100-bullet)

## Загрузка данных, предобработка, EDA

### Заявки (лиды)

In [2]:
leads_2 = pd.read_excel('Тестовое_задание_2.xlsx', sheet_name='Заявки')
leads_2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,sent,Курс,utm_source,NaN,NaN,NaN
1,2022-02-01 00:49:08,МПК,NaN,NaN,NaN,NaN
2,2022-02-01 11:15:09,СЯ,email,NaN,NaN,NaN
3,2022-02-01 11:31:24,СЯ,yandex,NaN,NaN,NaN
4,2022-02-01 13:19:38,РПП,NaN,NaN,NaN,NaN


In [3]:
leads_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  499 non-null    object 
 1   Unnamed: 1  499 non-null    object 
 2   Unnamed: 2  370 non-null    object 
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   Unnamed: 5  0 non-null      float64
dtypes: float64(3), object(3)
memory usage: 23.5+ KB


Всего с данными заявок 498 строк. В этой таблице у заявок указан только источник трафика (utm_source), но не во всех строках, то есть встречаются пропуски Nan-ы. Возвращаем "шапку" таблицы на первую строку, заполняем Nan-ы нулями.

In [4]:
# Удаляем пустые столбцы и первую строку:
leads_2 = leads_2.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis=1)
leads_2 = leads_2.drop([0], axis=0)

In [5]:
# Переименуем названия столбцов:
leads_2.set_axis(['date_time', 'course', 'utm_source'], axis = 'columns', inplace = True)

In [6]:
# Переводим дату в формат datetime:
leads_2['date_time'] = pd.to_datetime(leads_2['date_time'])
leads_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 1 to 498
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date_time   498 non-null    datetime64[ns]
 1   course      498 non-null    object        
 2   utm_source  369 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 15.6+ KB


In [7]:
# Заменяем пропуски нулями
leads_2['utm_source'] = leads_2['utm_source'].fillna(value='0')

In [8]:
leads_2.head()

,date_time,course,utm_source
1,2022-02-01 00:49:08,МПК,0
2,2022-02-01 11:15:09,СЯ,email
3,2022-02-01 11:31:24,СЯ,yandex
4,2022-02-01 13:19:38,РПП,0
5,2022-02-01 14:54:06,АСТ,0


За какое время собраны данные:

In [9]:
leads_2['date_time'].min()

Timestamp('2022-02-01 00:49:08')

In [10]:
leads_2['date_time'].max()

Timestamp('2022-02-28 22:34:57')

Данные с лидами собраны за период с 1 февраля по 28 февраля 2022 года.

In [11]:
# Выделим отдельно дату без времени
leads_2['date'] = leads_2['date_time'].dt.date
leads_2.sample(3)

,date_time,course,utm_source,date
213,2022-02-12 18:05:14,СЯ,partners,2022-02-12
69,2022-02-04 13:02:25,АСТ,0,2022-02-04
207,2022-02-12 15:38:29,СЯ,partners,2022-02-12


In [12]:
len(leads_2['course'].unique())

17

In [13]:
leads_2['course'].unique()

array(['МПК', 'СЯ', 'РПП', 'АСТ', 'ССТ', 'ОП', 'КП', 'МК', 'ЭФТ',
       'МПКПРО', 'HRM', 'ГП', 'АРТ', 'АСТР', 'НП', 'Логопед', 'ЮИКС'],
      dtype=object)

In [14]:
leads_2['utm_source'].unique()

array(['0', 'email', 'yandex', 'facebook', 'partners', 'google',
       'telegram', 'teachable', 'instagram', 'psychodemia', 'whatsapp',
       'igaryoh', 'b17', 'internal'], dtype=object)

In [15]:
len(leads_2['utm_source'].unique())

14

Заявки собраны на 17 курсов обучения, из 13 поименных источников трафика `utm_source`, и также есть заявки без указания источника.

Сколько было заявок на какие курсы:

In [16]:
leads_by_course = leads_2.groupby('course')[['date']].count().reset_index().rename(columns={"date": "leads"}).sort_values(by='leads', ascending = False)
leads_by_course

,course,leads
14,СЯ,111
12,РПП,92
8,МПК,75
13,ССТ,42
11,ОП,38
2,АСТ,34
5,КП,22
7,МК,16
9,МПКПРО,13
4,ГП,11


Больше всего заявок получено на курсы СЯ, РПП, МПК.

[Вернуться в начало](#100-bullet)

### Продажи

In [17]:
sales_2 = pd.read_excel('Тестовое_задание_2.xlsx', sheet_name='Продажи')

In [18]:
sales_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ДАТА        122 non-null    datetime64[ns]
 1   КУРС        122 non-null    object        
 2   ПОТОК       122 non-null    int64         
 3   ФИО         122 non-null    object        
 4   ПОЧТА       122 non-null    object        
 5   ТЕЛЕФОН     122 non-null    object        
 6   ПАКЕТ       122 non-null    object        
 7   СУММА       122 non-null    int64         
 8   МЕНЕДЖЕР    122 non-null    object        
 9   utm_source  32 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 9.7+ KB


In [19]:
sales_2.head()

,ДАТА,КУРС,ПОТОК,ФИО,ПОЧТА,ТЕЛЕФОН,ПАКЕТ,СУММА,МЕНЕДЖЕР,utm_source
0,2022-02-01,АСТ,2,Князева Марианна Ивановна,dwheeler@mac.com,7(1398)038-54-77,Стандартный,34766,Чумакова,google
1,2022-02-01,ССТ,6,Чернышев Макар Николаевич,mstrout@att.net,7(98)342-25-36,Расширенный,93579,Дружинина,telegram
2,2022-02-01,РПП,2,Никитин Даниил Александрович,crowemojo@icloud.com,7(4213)805-20-46,Стандартный,77670,Чумакова,NaN
3,2022-02-01,ССТ,3,Федорова Мелания Максимовна,hstiles@verizon.net,7(85)232-43-00,Стандартный,22689,Заможская,partners
4,2022-02-01,АСТ,1,Давыдов Андрей Макарович,rgarton@msn.com,7(5291)610-92-43,Стандартный,28174,Чумакова,NaN


Продаж 122 сделки, с форматами все ок, есть пропуски в источниках трафика `utm_source`.

In [20]:
# Заменяем пропуски нулями
sales_2['utm_source'] = sales_2['utm_source'].fillna(value='0')

In [21]:
sales_2['ДАТА'].min()

Timestamp('2022-02-01 00:00:00')

In [22]:
sales_2['ДАТА'].max()

Timestamp('2022-02-28 00:00:00')

Периоды, за которые собраны данные по заявкам и по продажам, полностью совпадают: это полный месяц февраль с 1 по 28 число, на некоторые сделки есть информация об источнике трафика. 

В представленных данных нет привязки конкретной Продажи к конкретной Заявке (нет id лида, id заявки).

In [23]:
sales_2['КУРС'].sort_values().unique()

array(['АСТ', 'КП', 'МК', 'МПК', 'МПКПРО ', 'ОП', 'РПП', 'ССТ', 'СЯ'],
      dtype=object)

In [24]:
len(sales_2['КУРС'].unique())

9

In [25]:
leads_2['course'].sort_values().unique()

array(['HRM', 'АРТ', 'АСТ', 'АСТР', 'ГП', 'КП', 'Логопед', 'МК', 'МПК',
       'МПКПРО', 'НП', 'ОП', 'РПП', 'ССТ', 'СЯ', 'ЭФТ', 'ЮИКС'],
      dtype=object)

Имеется 9 разновидностей курсов в "Продажах", это меньше, чем встречалось в "Заявках" (17), но все курсы из "Продаж" встречались в "Заявках".

Посчитаем, сколько продаж каждого курса случилось в феврале:

In [26]:
sales_by_course = sales_2.groupby('КУРС')[['ДАТА']].count().reset_index().rename(columns={"ДАТА": "ПРОДАЖИ"}).sort_values(by='ПРОДАЖИ', ascending = False)
sales_by_course

,КУРС,ПРОДАЖИ
6,РПП,28
7,ССТ,27
3,МПК,20
8,СЯ,18
5,ОП,14
0,АСТ,10
1,КП,2
2,МК,2
4,МПКПРО,1


Больше всего продали курсов с аббревиатурами РПП, ССТ, МПК. Из 122 строк с продажами источник трафика `utm_source` указан только у 32 сделок (четверть).

[Вернуться в начало](#100-bullet)

### Конверсии

Отсутствует привязка конкретной продажи к конкретной заявке (нет id пользователя, id заявки), чтобы понимать сроки конверсии (сколько времени от заявки до продажи), и выстраивать правильно когорты.

Но, и по заявкам, и по продажам есть данные за целый полный один и тот же месяц, и можно предложить гипотезу, что часть, и, возможно, бОльшая, часть продаж могла выйти из заявок, сделанных в течение одной-двух недель до продажи - эту гипотезу нужно проверять, собирая данные в большем объеме, а именно:
для того, чтобы разобраться, сколько требуется лиду времени от момента оставления заявки до момента покупки курса, и, вообще, для всего остального доказательного анализа, нужно каждой заявке (лиду), а лучше еще раньше - посетителю сайта с определенного IP, а лучше - всегда авторизовавшемуся посетителю, присваивать уникальный id, по которому его потом можно будет идентифицировать в течение его lifetime в этом продукте.

Сделаем группировки по дате, курсу, источнику, склеим и посмотрим на результаты.

Заявки:

In [27]:
leads_by_date_course_utm = leads_2.groupby(['date', 'course', 'utm_source'])[['date_time']].count().reset_index().rename(columns={"date_time": "leads"})
leads_by_date_course_utm['date'] = pd.to_datetime(leads_by_date_course_utm['date'])
leads_by_date_course_utm

,date,course,utm_source,leads
0,2022-02-01,АСТ,0,1
1,2022-02-01,АСТ,email,1
2,2022-02-01,КП,0,2
3,2022-02-01,МПК,0,2
4,2022-02-01,ОП,email,1
5,2022-02-01,РПП,0,1
6,2022-02-01,ССТ,facebook,1
7,2022-02-01,СЯ,0,1
8,2022-02-01,СЯ,email,1
9,2022-02-01,СЯ,yandex,2


Продажи:

In [28]:
sales_by_date_course_utm = sales_2.groupby(['ДАТА', 'КУРС', 'utm_source'])[['ПОТОК']].count().reset_index().rename(columns={"ПОТОК": "ПРОДАЖИ"})
sales_by_date_course_utm

,ДАТА,КУРС,utm_source,ПРОДАЖИ
0,2022-02-01,АСТ,0,2
1,2022-02-01,АСТ,google,1
2,2022-02-01,РПП,0,1
3,2022-02-01,ССТ,partners,1
4,2022-02-01,ССТ,telegram,1
5,2022-02-02,АСТ,0,3
6,2022-02-02,АСТ,telegram,1
7,2022-02-02,МПК,teachable,1
8,2022-02-02,ССТ,0,3
9,2022-02-02,СЯ,google,1


In [29]:
# "склеиваем" заявки и продажи через 'outer' по дате, курсу и источнику трафика utm_source
conv_by_date_course_utm = leads_by_date_course_utm.merge(sales_by_date_course_utm, left_on = ['date', 'course', 'utm_source'], right_on = ['ДАТА', 'КУРС', 'utm_source'], how='outer')
conv_by_date_course_utm['ПРОДАЖИ'] = conv_by_date_course_utm['ПРОДАЖИ'].fillna(value='0').astype('int')
conv_by_date_course_utm['conversion'] = (conv_by_date_course_utm['ПРОДАЖИ'] / conv_by_date_course_utm['leads']).round(2)
conv_by_date_course_utm.sort_values(by='ДАТА')

,date,course,utm_source,leads,ДАТА,КУРС,ПРОДАЖИ,conversion
0,2022-02-01,АСТ,0,1.0,2022-02-01,АСТ,2,2.00
370,NaT,NaN,telegram,NaN,2022-02-01,ССТ,1,NaN
369,NaT,NaN,partners,NaN,2022-02-01,ССТ,1,NaN
368,NaT,NaN,google,NaN,2022-02-01,АСТ,1,NaN
5,2022-02-01,РПП,0,1.0,2022-02-01,РПП,1,1.00
371,NaT,NaN,0,NaN,2022-02-02,АСТ,3,NaN
372,NaT,NaN,telegram,NaN,2022-02-02,АСТ,1,NaN
373,NaT,NaN,teachable,NaN,2022-02-02,МПК,1,NaN
374,NaT,NaN,google,NaN,2022-02-02,СЯ,1,NaN
20,2022-02-02,ССТ,0,1.0,2022-02-02,ССТ,3,3.00


In [30]:
conv_by_date_course_utm['leads'].sum()

498.0

In [31]:
conv_by_date_course_utm['ПРОДАЖИ'].sum()

122

Склеили через `outer`, проверили по числу заявок и продаж - склеилось всё правильно, но, как и предполагали, в большинстве случаев продажа происходит через какое-то время после заявки, их нельзя связать по дате заявки и дате продажи. Чтобы построить когорты и посмотреть конверсии, нужен идентификационный номер заявки, отслеживаемый в продаже.

Сделаем склейку без привязки к датам и источнику, просто целиком за месяц.

In [32]:
conv_by_course = leads_by_course.merge(sales_by_course, left_on = ['course'], right_on = ['КУРС'], how='left')
conv_by_course['ПРОДАЖИ'] = conv_by_course['ПРОДАЖИ'].fillna(value='0').astype('int')
conv_by_course['conversion'] = (conv_by_course['ПРОДАЖИ'] / conv_by_course['leads']).round(2)
conv_by_course.sort_values(by='conversion', ascending = False)

,course,leads,КУРС,ПРОДАЖИ,conversion
3,ССТ,42,ССТ,27,0.64
4,ОП,38,ОП,14,0.37
1,РПП,92,РПП,28,0.30
5,АСТ,34,АСТ,10,0.29
2,МПК,75,МПК,20,0.27
0,СЯ,111,СЯ,18,0.16
7,МК,16,МК,2,0.12
6,КП,22,КП,2,0.09
12,ЮИКС,6,NaN,0,0.00
15,АСТР,4,NaN,0,0.00


Если смотреть заявки и продажи за февраль, считая, что люди, отправившие заявку в феврале, тогда же купили курс, самая высокая конверсия у курсов ССТ (конверсия 0.64, то есть 27 продаж при 42 заявках) и ОП (конверсия 0.37, то есть 14 продаж при 38 заявках). У других курсов с высоким числом продаж было больше заявок, поэтому меньше конверсия: РПП (28 продаж на 92 заявки, конверсия 0.30), МПК (20 продаж на 75 заявок, конверсия = 0.27), АСТ (10 продаж на 34 заявки, конверсия = 0.29). Больше всего заявок было на курс СЯ = 111 заявок, но продаж всего 18, конверсия = 0.16. Курс КП - продаж 2, заявок 22, конверсия = 0.09.

Среди источников трафика `utm_source`, по которым можно отследить наибольшую конверсию, лидируют `teachable`, `partners` и `telegram`.

[Вернуться в начало](#100-bullet)

## Динамические отчеты

Выгрузим откорректированные файлы с данными, чтобы к ним обращаться из Tableau:

In [33]:
leads_2.to_csv('leads_2.csv', index = False)

In [34]:
sales_2.to_csv('sales_2.csv', index = False)

In [35]:
conv_by_course.to_csv('conversion_2.csv', index = False)

Динамические отчеты в Tableau: https://public.tableau.com/app/profile/mvavdonina/viz/Psychodemia_2/Online_courses

[Вернуться в начало](#100-bullet)

## Выводы

Предоставлены данные о заявках с сайта (лидах) и фактических продажах курсов онлайн-школы психологических профессий. Цель сайта: лидогенерация. Сбор лидов идет через форму "Оставить заявку", также можно написать в чат, позвонить по телефону или обратиться на имейл.

В "Заявках" собраны данные о дате и времени заявки, названии курса и источнике трафика `utm_source`, который указан для большинства заявок, но не для всех. Период сбора заявок: с 1 по 28 февраля 2022. Всего заявок 498. Представлено 17 разновидностей курсов обучения, 13 поименных источников трафика `utm_source`, и есть заявки без указания источника. Больше всего заявок на курсы СЯ, РПП, МПК.

В "Продажах" собраны данные о дате продажи, менеджере, осуществившем продажу, названии курса, контактных данных покупателя (фио, почта, телефон), потоке, пакете, сумме, а также источнике трафика `utm_source` для части продаж. Период продаж: с 1 по 28 февраля 2022 - совпадает с периодом заявок. Всего продаж 122. 
В "Продажах" зафиксировано 9 разновидностей курсов, все курсы из "Продаж" встречались в "Заявках". Больше всего продаж курсов РПП, ССТ, МПК. Из 122 сделок с продажами источник трафика `utm_source` указан только у 32 продаж из 122 (четверть).

Отсутствует привязка конкретной продажи к конкретной заявке (нет id пользователя, id заявки), чтобы понимать сроки конверсии (сколько времени от заявки до продажи) и выстраивать правильно когорты.
Но, и по заявкам, и по продажам есть данные за полный целый один и тот же месяц, и есть вероятность, что часть, и, возможно, бОльшая, продаж могла выйти из заявок, сделанных в течение одной-двух недель до продажи (гипотеза - нужно проверять).

При склейке всех заявок со всеми продажами по датам в разрезе курсов и источников трафика полностью (через `outer`), подтвердили свое предположение о том, что в большинстве случаев продажа происходит через какое-то время после заявки (не в тот же день), их нельзя связать по дате заявки и дате продажи. Чтобы построить когорты и посмотреть конверсии точно, нужен идентификационный номер заявки, отслеживаемый в продаже.

При склейке данных за месяц по курсам без привязки к точным датам и источнику трафика в предположении, что люди, отправившие заявку в феврале, тогда же купили курс, обнаружили следующие результаты:
- самая высокая конверсия у курсов ССТ (конверсия 0.64, то есть 27 продаж при 42 заявках) и ОП (конверсия 0.37, то есть 14 продаж при 38 заявках);
- других курсов с высоким числом продаж было больше заявок, поэтому меньше конверсия: РПП (28 продаж на 92 заявки, конверсия 0.30), МПК (20 продаж на 75 заявок, конверсия = 0.27), АСТ (10 продаж на 34 заявки, конверсия = 0.29); 
- больше всего заявок было на курс СЯ = 111 заявок, но продаж всего 18, конверсия = 0.16;
- наименьшая конверсия у курса КП  = 0.09, заявок 22, продаж 2.

Среди источников трафика `utm_source`, по которым можно отследить наибольшую конверсию, лидируют `teachable`, `partners` и `telegram`.

Динамические отчеты в Tableau: https://public.tableau.com/app/profile/mvavdonina/viz/Psychodemia_2/Online_courses

Рекомендация: ставить идентификаторы на заявки для отслеживания их дальнейшей жизни в проекте.

[Вернуться в начало](#100-bullet)